In [1]:
# PART OF THIS CODE IS FROM MICHAEL MURPHY - THANKS!
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from collections import OrderedDict
import glob
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, sys
import glob, re
import seaborn as sns
pd.set_option('display.max_rows', 500) 
pd.set_option('display.max_columns', 100)

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score, train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.utils.multiclass import type_of_target # used to check the Y labels are appropriate for classification
from sklearn.metrics import roc_curve
from sklearn import metrics
from sklearn.utils import shuffle
from scipy import interp

In [3]:
#### Use this if DOING a fresh modeling fitting analysis
bn = True # use the percentile normalized data or no? 
log = False
stand_scaler = False
reanalysis = True
model = 'log_reg' #log_reg, rf or svm

if reanalysis:
    pre_norm_ds = [ 'plasmaall_author',
                    'urineall_author',
                    'm_oxylipin_chronic_hep_b',
                    'm_chronic_hep_b_POS',
                    'm_chronic_hep_b_NEG',
                    'm_CER_mass_spectrometry_v4',
                    'm_CER_mass_spectrometry_v4_3_CS',
                    'm_CER_mass_spectrometry_v4_0_NS',
                    'm_CER_mass_spectrometry_v4_2_FS',
                    'm_CER_mass_spectrometry_v4_1_COPD',
                    'm_EICO_mass_spectrometry_v4',
                    'm_EICO_mass_spectrometry_v4_3_CS',
                    'm_EICO_mass_spectrometry_v4_0_NS',
                    'm_EICO_mass_spectrometry_v4_2_FS',
                    'm_EICO_mass_spectrometry_v4_1_COPD',
                    'AN000580',
                    'AN000581',
                    'AN001503']

    if bn:
#         path = './bn_pickles/*.pkl'
#         path = './bn_pickles/ST00006*_bn_data.pkl'
        path = './bn_pickles/MTBLS315*.pkl'
    else:
        path = './pickles/*.pkl'

    datasets = OrderedDict()
    for fn in sorted(glob.glob(path)):
        data = pd.read_pickle(open(fn,'rb'))
        datasets[data[0]['study']] = data
    
else:
    #### Use this if NOT doing a fresh modeling fitting analysis
    pickle_file = './YES_bn_ds_models_and_sigfeat_NO_log_NO_standscal_NO_multi_mapped_labels.pkl'
    ### The non-batch corrected pickle for the dataset
    # pickle_file = './NO_bn_dataset_models_and_sigfeat_YES_log.pkl'
    datasets = pickle.load(open(pickle_file, 'rb'))

In [6]:
for k,v in datasets.items():
    for ds in v:
        print(ds['data_set'])

m_GC_nmfi_and_bsi_diagnosis_v2_maf
m_LC_nmfi_and_bsi_diagnosis_v2_maf
m_UPLC_NEG_nmfi_and_bsi_diagnosis_v2_maf
m_UPLC_POS_nmfi_and_bsi_diagnosis_v2_maf
IPO_aligned_MTBLS315_mzData
IPO_aligned_MTBLS315_mzXML
IPO_aligned_MTBLS315_n_mzML
IPO_aligned_MTBLS315_p_mzML


In [43]:
seed = 981432
def convert_nan_to_val(data, value=0):
    data[pd.isnull(data)] = value
    return data

def get_reduced_data(data_list):
    peaks = data_list['peaks'].values.T
#     print(peaks)
    data = data_list['features']
    combined = np.concatenate((peaks,data)).T
#     combined = combined[:,1:]
#     print(combined)
    # not its mz/rt/number_samples x feat
    # sort be the rt
    combined = combined.view('f32,f32,d32').sort(order=['f1'], axis=0)
#     combined = np.sort(combined, axis=0)

base_fpr = np.linspace(0,1,101)
for k, v in datasets.items():
    for dataset in v:
        tprs = []
        X = dataset['features'].values.copy()
        # do data transformation and 0 filling 
        if 'IPO' in dataset['data_set']:
            ds = dataset['data_set']
            X = convert_nan_to_val(X, value=1)
            X = X[:,dataset['peaks']['X1']>15]
        else:
    #         continue
            ds = dataset['data_set']
            X[X == 0] = 1
    ### Filter features based on having a mean feature value greater than a certain value:
    #     X = X[:,X.mean(0) > 100]

#         X = np.log2(X)
        y = dataset['labels'].values.copy().ravel().astype(int)

    ####working and looks good
    #     x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3) 
    #     scaler = StandardScaler()
    #     x_train = scaler.fit_transform(x_train)
    #     x_test = scaler.transform(x_test)
    #     clf = LogisticRegressionCV(penalty='l1', solver='liblinear', tol=1e-4, intercept_scaling=10, max_iter=500)
    #     clf.fit(x_train, y_train)
    #     y_pred = clf.predict_proba(x_test)
    #     fpr, tpr, _ = roc_curve(y_test, y_pred[:,1])
    #     auc_values = metrics.auc(fpr, tpr)
    #     plt.plot(fpr, tpr, 'b', label='AUC = %0.2f' % auc_values)
    #     plt.legend(loc='lower right')
    #     plt.show()
#         get_reduced_data(dataset)
        X,y = shuffle(X,y)
        clf = LogisticRegressionCV(penalty='l1', solver='liblinear', tol=1e-4, intercept_scaling=10, max_iter=500)
        cv = StratifiedKFold(n_splits=3, shuffle=True)
        aucs = []
        for train, test in cv.split(X,y):
            x_train, y_train = X[train], y[train]
            x_test,y_test = X[test], y[test]
            scaler = StandardScaler()
            x_train = scaler.fit_transform(x_train)
            x_test = scaler.transform(x_test)
            clf.fit(x_train, y_train)
            y_pred = clf.predict_proba(x_test)
            fpr, tpr, _ = roc_curve(y_test, y_pred[:,1])
            auc_value = metrics.auc(fpr, tpr)
            aucs.append(auc_value)
    #         plt.plot(fpr, tpr, 'b', alpha=0.15) #, label='AUC = %0.2f' % auc_values)
            tpr = interp(base_fpr, fpr, tpr)
            tpr[0] = 0.0 #just to make it start at 0
            tprs.append(tpr)
    
        tprs = np.array(tprs)
        mean_tprs = tprs.mean(axis=0)
        std = tprs.std(axis=0)
        tprs_upper = np.minimum(mean_tprs + std, 1)
        tprs_lower = mean_tprs - std
        aucs = np.asarray(aucs)
        print(ds, aucs.mean(), aucs.std())
        import matplotlib as mpl
        mpl.rcParams['axes.linewidth'] = 1.5
        plt.figure(figsize=(8,6))
        plt.plot(base_fpr, mean_tprs, 'b',linewidth=4.0, label='Mean ROC - AUC = %0.2f $\pm$ %0.2f' %(aucs.mean(),aucs.std()))
        plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3, label='$\pm$ 1 stdev')
        plt.plot([0, 1], [0, 1],'r--', linewidth=1.0, label='Luck') 
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.tick_params(axis='both', which='major', labelsize=16)
        plt.xlabel('False Positive Rate', fontsize=17)
        plt.ylabel('True Positive Rate', fontsize=17)
        plt.title('Malaria vs. Non-malarial fevers:' +'\n'+'Full Retention time', fontsize=25)
    #     plt.title('ROC analysis for %s' % ds)
        leg = plt.legend(loc='lower right', fontsize=16, framealpha=0.3)
    #     for line in leg.get_lines():
    #         line.set_linewidth(1)
        ax = plt.gca()
        ax.get_xaxis().set_tick_params(direction='out', width=1)
        ax.get_yaxis().set_tick_params(direction='out', width=1)
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
    #     plt.show() 
        plt.tight_layout()
        plt.savefig('Malaria_v_non-malaria: all feature_%s.png' % ds)
        plt.gcf().clear()
    
    
    
#     perform model training on each split of the data   
#     cv = StratifiedKFold(n_splits=3)
#     aucs = []
#     for train, test in cv.split(X,y):
#         scaler = StandardScaler()
# #         clf = LogisticRegression(penalty='l1', solver='liblinear', tol=1e-4, C=0.1,
# #                                                         intercept_scaling=10, max_iter=500)
#         clf = GridSearchCV(estimator=LogisticRegression(penalty='l1', solver='liblinear', tol=1e-4, 
#                                                         intercept_scaling=10, max_iter=500),
#                            cv=StratifiedKFold(3, random_state=seed+1), param_grid={'C': 10.**np.linspace(-4,4)},
#                            n_jobs=-1, scoring='roc_auc') 
#         X_d = scaler.fit_transform(X[train])
#         X_t = scaler.transform(X[test])
#         clf.fit(X_d, y[train])
#         y_pred = clf.predict_proba(X_t)
#         fpr, tpr, _ = roc_curve(y[test], y_pred[:,1])
#         auc_values = metrics.auc(fpr, tpr)
#         aucs.append(auc_values)
# #         plt.plot(fpr, tpr, 'b', alpha=0.15, label='AUC = %0.2f' % auc_values)
#         tpr = interp(base_fpr, fpr, tpr)
#         tpr[0] = 0.0 #just to make it start at 0
#         tprs.append(tpr)
# #         print(fpr)
# #         print(tpr)
# #         print(y[test])
# #         print(y_pred[:,1])
#     tprs = np.array(tprs)
#     mean_tprs = tprs.mean(axis=0)
#     std = tprs.std(axis=0)
#     tprs_upper = np.minimum(mean_tprs + std, 1)
#     tprs_lower = mean_tprs - std
#     aucs = np.asarray(aucs)
    
    
    
#     plt.plot(base_fpr, mean_tprs, 'b', label='AUC = %0.2f +/- %0.2f' %(aucs.mean(),aucs.std()))
#     plt.fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)
#     plt.plot([0, 1], [0, 1],'r--')
#     plt.xlim([-0.01, 1.01])
#     plt.ylim([-0.01, 1.01])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('ROC analysis for %s' % ds)
#     plt.legend(loc='lower right')
#     plt.show()

m_GC_nmfi_and_bsi_diagnosis_v2_maf 0.7746913580246915 0.03327726392647142
m_LC_nmfi_and_bsi_diagnosis_v2_maf 0.7550505050505051 0.09903369470266453
m_UPLC_NEG_nmfi_and_bsi_diagnosis_v2_maf 0.8473625140291808 0.05833980434732669
m_UPLC_POS_nmfi_and_bsi_diagnosis_v2_maf 0.893939393939394 0.05455168938053755
IPO_aligned_MTBLS315_mzData 0.5238496071829406 0.06886713176622179
IPO_aligned_MTBLS315_mzXML 0.8577441077441078 0.05233768200912393
IPO_aligned_MTBLS315_n_mzML 0.9424803591470258 0.03370041651991214
IPO_aligned_MTBLS315_p_mzML 0.9214365881032548 0.02494512993366204


<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>

<Figure size 576x432 with 0 Axes>